# Introduction

This is an introductory, hands-on, workshop for programming GPU devices using [CUDA](https://developer.nvidia.com/cuda-zone) C/C++. Even though we will be using a Jupyter notebook to edit and run our code be adviced that none of the commands are actually executed here but rather on the server (be it one of the cloud instances or your local machine).

In this lab we will:

1. see a brief overview of CPU vs GPU architectures
2. discuss why GPUs are better for certain tasks often utilized in machine learning and what are the limitations
3. explain how to extend C/C++ code using CUDA so it can be executed on GPUS
4. write simple GPU enabled programs
5. mention several higher level libraries, which will make you more productive

---

For people not acquainted with how Jupyter Notebook works, every notebook consists of a set of *cells*, each having a type. Each cell can be executed by navigating to it (e.g. by clicking it) and pressing **ctrl+enter** (or **cmd+enter** on macbooks) or by clicking the "Run cell" icon in the top menu.

Try and run the below cell (you can run terminal commands in a *code* cell by prepending them with an exclamation **!** mark). The output, containing information about GPU devices running on the server, should appear below it.

In [ ]:
!nvidia-smi

# CPU vs GPU Overview

![cpu-gpu-transistors](gpu-devotes-more-transistors-to-data-processing.png)

**CPU**
* few cores optimized for *serial processing*
* lower memory bandwith (but direct access to more memory) 
* *latency* optimized cores (faster at a single task but can only perform few at the same time)
* more instructions but slower execution

**GPU**
* hundreds/thousands of smaller, more efficient cores optimized for *multiple tasks simultaneously*
* great for compute-intensive parts of the application
* higher memory bandwidth (but direct access to less memory than CPU)
* *throughput* optimized cores (slower at single tasks but can perform more at the same time)
* limited number of instructions available but faster execution

![how-gpu-acceleration-works](how-gpu-acceleration-works.png)
*http://www.nvidia.com/object/what-is-gpu-computing.html

In a GPU accelerated appplication the compute-intensive parts are offloaded to the GPU, while the remaining of the application runs on the CPU.

## GPUs in Machine Learning

**Pros**
* ML algorithms tend to be **compute-bound** (require more computations than fetches from memory) - GPUs are intended for compute intensive tasks!
* many ML algorithms consist of highly parallelizable tasks e.g. matrix multiplications peformed over multiple iterations - GPUs are great at those

**Drawbacks**
* host RAM to GPU RAM memory copy overhead (over PCI-E)
* limited (very limited compared to host RAM) GPU RAM size
* serial tasks slow

# CUDA Overview

Here we will outline the main concepts behind the CUDA programming model (focusing on how they are exposed in C). A more in-depth explanation can be found in the [official programming guide](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html).

**host** - CPU and RAM

**device** - GPU and it's RAM

**kernels** - special functions, which can be called from host code (regular C code running on the CPU) but are run on the device (GPU) N times in parallel, executed by N CUDA threads.

**\__global__** - special CUDA C keyword used as part of a method signature to mark that method as a kernel.

**\__host__** - methods marked with this keyword can be only called from host code and will also run on the host

**\__device__** - methods marked with this keyword can be only called from device code and will also run on the device

![grid-of-thread-blocks](grid-of-thread-blocks.png)

**thread block** - CUDA programming model follows a well defined thread hierarchy model in which threads during execution are grouped into so called thread blocks. Thread blocks can be one, two or three dimensional. This very naturally maps to vectors, matrices and volumes.

There's a limit of threads per block since 1) all the threads in a block are expected to reside on the same processor core 2) have to share very limited resources of said core.

We can run a kernel, though, using multiple blocks each with the same number of threads. The total number of threads per kernel in such a case will be number_of_blocks * number_of_threads_per_block.

**threadIdx.{x,y,z}** - similar to the above these variables identify the thread ID that is being executed with the current block.

**blockIdx.{x,y,z}** - these 3 special, read-only, variables can be used within CUDA kernels to find out which block is executing the code at a given time. This, and the following read-only variables, are usually used to identify which part of data should be handled by a given kernel instance as there will be N of them running in parallel.

**blockDim.{x,y,z}** - another special read-only variable accesible within CUDA kernels. Contrary to the previous ones these are constant and describe the total number of threads within a block.

**grid** - similarly to threads being organized in thread blocks, thread blocks are organized into one, two or three dimensional grids. The number of blocks is dictated by the size of data and/or number of processors available.

**methodName <<< blocks, threads_per_block>>> (parameters)** - this is how you launch a kernel. You use this syntax in your host code (regular C code which runs on the CPU) but it is executed on the device (GPU). There are numerous variants of this operator but the most basic one takes in 2 numbers: the number of blocks to run on the GPU and the second the number of threads per each block.

# Lab 0: Hello CUDA

Lets start with the most basic C program there is, a "Hello, World!". We defined one in the [lab0/lab0a.c](/edit/lab0/lab0a.c) file. You can run terminal commans from this notebook by prepending them with an exclamation mark **!**. The command below will compile the C file using **gcc**, which will output the **lab0/lab0a.out** binary and subsequently run it:

In [ ]:
!gcc lab0/lab0a.c -o lab0/lab0a.out && lab0/lab0a.out 

Now lets modify this code so it can run on the GPU. The modified version can be found here [lab0/lab0b.cu](/edit/lab0/lab0b.cu) (.cu being the usual suffix for CUDA source files). As you can see several things changed:

1. the **hello** function has been prefixed with the **__global__** keyword making it a kernel
2. thanks to the above we can call threadIdx.x and blockIdx.x (and other variables discussed before) to get additional information on the context in which the kernel is being ran
3. the call to the *hello* method has been changed to the **<<<...>>>** syntax - in this case we will run it only using 1 block and 1 thread.
4. a call to **cudaDeviceSynchronize()** has been added as all calls to kernels are **asynchronous**, meaning our main method might return before anything got actually printed. This method is blocking and makes sure all the operations on the GPU finish before it returns.

In [ ]:
!nvcc -o lab0/lab0b.out lab0/lab0b.cu -run

**Try** changing the values (modify the file, save it and rerun the above cell) between **<<<...>>>** to:

- **<<< 2, 1 >>>**
- **<<< 1, 32 >>>** how did the output change? Is the order what you'd expect?
- **<<< 2, 16 >>>** can you set those numbers much higher?

# Lab 1: Adding arrays

Now that you know what kernels are, how to write and launch them lets try to write one which adds two arrays together storing the result in a 3rd array.

## Custom kernel

We already prepared all the boilerplate code for you in [lab1/lab1a.cu](/edit/lab1/lab1a.cu) so please just focus on the parts marked by TODO comments. You will notice certain pieces of code in the main method which we will cover in later labs, please ignore them for now.

You'll have to figure out 3 things:

- modify a regular C signature into a CUDA kernel
- use the read-only CUDA provided variables (described in the CUDA overview section) to calculate the index for each kernel instance
- in our example we will have number_of_blocks * threads_per_block instances, what if this number if bigger than the sizes of our arrays? You will need to sanity check this

After modifying the file be sure to save it either by pressing **ctrl+s** (**cmd+s**) or using the top menu File -> Save. When you're done run the cell below. Your result should show all 0s.

In [ ]:
!nvcc -o lab1/lab1a.out lab1/lab1a.cu -run

If you're having problems you can have a look at the solution [lab1/lab1a_solution.cu](/edit/lab1/lab1a_solution.cu).

## Thrust

Quoting the official [Thrust website](https://developer.nvidia.com/thrust):

> Thrust is a powerful library of parallel algorithms and data structures. Thrust provides a flexible, high-level interface for GPU programming that greatly enhances developer productivity. Using Thrust, C++ developers can write just a few lines of code to perform GPU-accelerated sort, scan, transform, and reduction operations orders of magnitude faster than the latest multi-core CPUs.

Thrust follows very closely the [C++ Standard Template Library](http://www.cplusplus.com/reference/stl/) API and approach exposing high level access to many useful containers and algorithms. It makes moving data between host and device memories very easy and provided algorithms are highly optimized for GPUs (taking into account things like nondeterministic order of operations and working with floating point precision numbers).

In [lab1/lab1b.cu](/edit/lab1/lab1b.cu) we use [Thrust vectors](http://docs.nvidia.com/cuda/thrust/index.html#vectors) for the same problem as before. Thrust device_vector makes it easy to allocate memory on the device and assign values. Moving data back to host memory is easily accomplished by declaring host_vector and assigning device_vector to it.

In this exercise we want to use one of the [Thrust transformation methods](https://thrust.github.io/doc/group__transformations.html). We can think about vector addition as a transformation of 2 iterators where we sum appropriate elements of each vector and put the result into the output vector.

After modifying the file you can execute the cell below to run it. You should get the same output as in the previous exercise. All necessary includes are already in the file, focus just on the TODO part.

In [ ]:
!nvcc -o lab1/lab1b.out lab1/lab1b.cu -run

[Hint1](/edit/lab1/hint1.txt)

[Hint2](/edit/lab1/hint2.txt)

If you still find it challenging you can check the [solution here](/edit/lab1/lab1b_solution.cu)

# Lab 2: Matrix multiplication

## Custom kernel

## cuBLAS

# Lab 3: Error handling

# Lab 4: Memory management

# Lab 5: Thread synchronization